In [15]:
import sys
import pandas as pd
sys.path.insert(1, '/Users/max/projects/machine-learning-labs/tensorflow')
from mytensorflow import *
import tensorflow as tf
import numpy as np

import tensorflow_datasets as tfdt
import pandas as pd
import numpy as np
import os
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model

from  matplotlib import pyplot as plt
import random
import tensorflow as tf
import os
from PIL import Image
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from tensorflow import keras
import matplotlib.image as mpimg
%matplotlib inline

In [16]:
IMG_WIDTH=200
IMG_HEIGHT=200
img_folder=r'/Users/max/projects/machine-learning-labs/data/catsdogs'

In [17]:
import os
def create_dataset_PIL(img_folder):
    
    img_data_array=[]
    class_name=[]
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= np.array(Image.open(image_path))
            image= np.resize(image,(IMG_HEIGHT,IMG_WIDTH,3))
            image = image.astype('float32')
            image /= 255  
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array , class_name  

In [18]:
PIL_img_data, class_name=create_dataset_PIL(img_folder)

In [19]:
target_dict={k: v for v, k in enumerate(np.unique(class_name))}

In [20]:
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]

In [21]:
net = Network()
net.add(ConvolutionalLayer((200,200,3), (3,3), 1))
net.add(FlattenLayer())
net.add(FCLayer(39204, 1))
net.add(ActivationLayer(Sigmoid.callFunction, Sigmoid.callDerivative))
net.compile(BinaryCrossEntropy.fun, BinaryCrossEntropy.derivative)
net.fit(np.array(PIL_img_data[::1000]), tf.cast(list(map(int,target_val[::1000])),tf.int32), epochs=50, learning_rate=0.1)

ValueError: operands could not be broadcast together with shapes (200,200) (199,199) (200,200) 

In [ ]:
img = tf.keras.preprocessing.image.load_img(
    "../data/catsdogs/cats/0.jpg", target_size=(200, 200)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = net.predict(img_array)
predictions

[array([0.49717491])]